In [2]:
# Install required packages
!pip install --upgrade pip
!pip install ultralytics opencv-python kagglehub huggingface-hub pillow torch torchvision pyyaml

# Verify installations
import ultralytics
print(f"YOLOv8 version: {ultralytics.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLOv8 version: 8.4.12


In [ ]:
import os
import getpass
from google.colab import files

print("\n" + "="*60)
print("KAGGLE AUTHENTICATION")
print("="*60)
print("\nChoose your authentication method:")
print("  1. Paste Kaggle API Token (RECOMMENDED)")
print("  2. Upload kaggle.json file")
print("\nYour token will NOT be saved or displayed.")

choice = input("\nEnter choice (1 or 2): ").strip()

if choice == "1":
    print("\n📝 Enter your Kaggle API token")
    print("   Get it from: https://www.kaggle.com/settings/account")
    print("   Click 'Create New API Token' and copy the token value\n")

    kaggle_token = "xxxxx"

    if not kaggle_token:
        raise ValueError("Token cannot be empty!")

    os.environ['KAGGLE_API_TOKEN'] = kaggle_token

    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

    print("\nKaggle credentials configured using API token!")
    print("   Token is stored in memory for this session only.")

elif choice == "2":

    uploaded = files.upload()

    if 'kaggle.json' not in uploaded:
        raise FileNotFoundError("kaggle.json file not found!")

    # Setup Kaggle
    os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    print("\nKaggle credentials configured using kaggle.json!")
else:
    raise ValueError("Invalid choice. Please enter 1 or 2.")

print("\n" + "="*60)


KAGGLE AUTHENTICATION

Choose your authentication method:
  1. Paste Kaggle API Token (RECOMMENDED)
  2. Upload kaggle.json file

Your token will NOT be saved or displayed.

Enter choice (1 or 2): 1

📝 Enter your Kaggle API token
   Get it from: https://www.kaggle.com/settings/account
   Click 'Create New API Token' and copy the token value


Kaggle credentials configured using API token!
   Token is stored in memory for this session only.



In [5]:
import kagglehub
import os

print("\n Downloading suitcase dataset from Kaggle...")

try:
    dataset_path = kagglehub.dataset_download("dataclusterlabs/handbag-image-dataset-luggage-dataset")
    print(f"\nDataset downloaded successfully!")
    print(f"Path: {dataset_path}")
except Exception as e:
    print(f"\nError downloading dataset: {str(e)}")
    print("\nTroubleshooting:")
    print("  1. Verify your Kaggle API token is correct")
    print("  2. Check that you have access to the dataset")
    print("  3. Ensure your internet connection is stable")
    raise

print("\nDataset contents:")
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        file_count = len(os.listdir(item_path))
        print(f"  {item}: {file_count} files")
    else:
        print(f"  {item}")

100%|██████████| 112M/112M [00:00<00:00, 214MB/s] 

Extracting files...



Dataset downloaded successfully!
Path: /root/.cache/kagglehub/datasets/dataclusterlabs/handbag-image-dataset-luggage-dataset/versions/1

Dataset contents:
  hand bag 300: 82 files


In [14]:
!ls /content/yolo_dataset/images/val | head


In [6]:
import os
import random
import shutil
from pathlib import Path
import xml.etree.ElementTree as ET

# ---------------- CONFIG ----------------
SOURCE_DIR = r"/root/.cache/kagglehub/datasets/dataclusterlabs/handbag-image-dataset-luggage-dataset/versions/1/hand bag 300"
# folder with jpg + xml together
YOLO_ROOT = "/content/yolo_dataset"
TRAIN_RATIO = 0.8

CLASSES = {
    "hand_bag": 0
}

# ----------------------------------------

def convert_voc_to_yolo(xml_path, txt_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    size = root.find("size")
    img_w = float(size.find("width").text)
    img_h = float(size.find("height").text)

    lines = []

    for obj in root.findall("object"):
        cls_name = obj.find("name").text.strip()

        if cls_name not in CLASSES:
            continue

        cls_id = CLASSES[cls_name]
        bnd = obj.find("bndbox")

        xmin = float(bnd.find("xmin").text)
        ymin = float(bnd.find("ymin").text)
        xmax = float(bnd.find("xmax").text)
        ymax = float(bnd.find("ymax").text)

        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        width = (xmax - xmin) / img_w
        height = (ymax - ymin) / img_h

        lines.append(
            f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        )

    with open(txt_path, "w") as f:
        f.write("\n".join(lines))


# ---------------- CREATE DIRS ----------------
for split in ["train", "val"]:
    os.makedirs(f"{YOLO_ROOT}/images/{split}", exist_ok=True)
    os.makedirs(f"{YOLO_ROOT}/labels/{split}", exist_ok=True)

# ---------------- FIND FILES ----------------
images = sorted([f for f in os.listdir(SOURCE_DIR) if f.lower().endswith(".jpg")])
random.shuffle(images)

split_idx = int(len(images) * TRAIN_RATIO)
train_imgs = images[:split_idx]
val_imgs = images[split_idx:]

def process_split(image_list, split):
    for img_name in image_list:
        base = Path(img_name).stem
        img_src = f"{SOURCE_DIR}/{img_name}"
        xml_src = f"{SOURCE_DIR}/{base}.xml"

        if not os.path.exists(xml_src):
            print(f"⚠️ Missing XML for {img_name}")
            continue

        img_dst = f"{YOLO_ROOT}/images/{split}/{img_name}"
        txt_dst = f"{YOLO_ROOT}/labels/{split}/{base}.txt"

        shutil.copy(img_src, img_dst)
        convert_voc_to_yolo(xml_src, txt_dst)

process_split(train_imgs, "train")
process_split(val_imgs, "val")

print("✅ VOC → YOLO conversion complete")
print(f"Train images: {len(train_imgs)}")
print(f"Val images: {len(val_imgs)}")


✅ VOC → YOLO conversion complete
Train images: 32
Val images: 9


In [7]:
import os

def print_tree(path, indent=0, max_depth=4):
    if indent > max_depth:
        return
    for item in sorted(os.listdir(path)):
        full = os.path.join(path, item)
        print("  " * indent + "├── " + item)
        if os.path.isdir(full):
            print_tree(full, indent + 1)

print_tree("/content/yolo_dataset")


├── images
  ├── train
    ├── 20220805_19_41_48_785_000_ht3IhIFrQCZrqoxy6s6pNmKMiSB2_F_3000_4000.jpg
    ├── 20220806_13_35_44_229_000_bRdv1ZdJyuNZ1KHjHqI2DLGVsz22_F_3000_4000.jpg
    ├── 20220806_13_35_59_540_000_bRdv1ZdJyuNZ1KHjHqI2DLGVsz22_F_3000_4000.jpg
    ├── 20220806_13_38_22_256_000_bRdv1ZdJyuNZ1KHjHqI2DLGVsz22_F_3000_4000.jpg
    ├── 20220806_13_45_19_589_000_bRdv1ZdJyuNZ1KHjHqI2DLGVsz22_F_3000_4000.jpg
    ├── 20220806_21_26_28_955_000_H9qFRyQ2E1h23Ll2o7AlWj4uass2_F_3468_4624.jpg
    ├── 20220806_21_27_29_003_000_H9qFRyQ2E1h23Ll2o7AlWj4uass2_F_3468_4624.jpg
    ├── 20220807_20_40_10_974_000_wakdSfqhnYYxaa1gBZGSEcwTk7F2_F_4000_3000.jpg
    ├── 20220807_20_41_31_277_000_wakdSfqhnYYxaa1gBZGSEcwTk7F2_F_4000_3000.jpg
    ├── 20220808_11_49_09_592_000_XJiUZI2Ge3TPX7tFsTFkzqaeno43_F_4032_3024.jpg
    ├── 20220808_13_28_31_170_000_JcobVtf6pxR3lw1dSehazS43kvq2_F_4160_3120.jpg
    ├── 20220810_11_21_35_159_000_Pb89FiEqcmXDf1ogn0tB6LzxDgI2_F_3264_2448.jpg
    ├── 20220810_11_22_04_190

In [8]:
# Path where the YAML file will be created
yaml_path = "/content/handbag.yaml"

yaml_content = """
path: /content/yolo_dataset

train: images/train
val: images/val

nc: 1
names:
  - hand_bag
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())

print(f"✅ YOLO dataset YAML created at: {yaml_path}")


✅ YOLO dataset YAML created at: /content/handbag.yaml


In [8]:
!cat /content/handbag.yaml


path: /content/yolo_dataset

train: images/train
val: images/val

nc: 1
names:
  - hand_bag

In [ ]:
from huggingface_hub import login
import getpass

print("\n" + "="*60)
print("🤗 HUGGING FACE AUTHENTICATION")

hf_token ="xxxxx"

# Login to Hugging Face
try:
    login(token=hf_token)
    print("\n  Logged in to Hugging Face successfully!")
except Exception as e:
    print(f"\n  Authentication failed: {str(e)}")
    raise



from huggingface_hub import hf_hub_download
from ultralytics import YOLO

REPO_ID = "IndUSV/Yolov8_SE_2"
FILENAME = "Yolov8_SE_2.pt"   # ⚠️ must be a YOLO .pt file

model_path = hf_hub_download(
    repo_id=REPO_ID,
    filename=FILENAME
)

print("✅ Downloaded model to:", model_path)




🤗 HUGGING FACE AUTHENTICATION

  Logged in to Hugging Face successfully!


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Yolov8_SE_2.pt:   0%|          | 0.00/6.23M [00:00<?, ?B/s]

✅ Downloaded model to: /root/.cache/huggingface/hub/models--IndUSV--Yolov8_SE_2/snapshots/feaf5ccb8aa55316e0dbf3add653a3ce5bfe3353/Yolov8_SE_2.pt


In [10]:
from ultralytics import YOLO

model = YOLO(model_path)

print("\n" + "="*60)
print("STARTING MODEL TRAINING")
print("="*60 + "\n")

epochs = 50
imgsz = 640
batch_size = 16
patience = 10

print("Training configuration:")
print(f"  • Model: YOLOv8n")
print(f"  • Epochs: {epochs}")
print(f"  • Image size: {imgsz}")
print(f"  • Batch size: {batch_size}")
print(f"  • Early stopping patience: {patience}")
print(f"  • Dataset YAML: {yaml_path}\n")

results = model.train(
    data=yaml_path,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    patience=patience,
    device="cpu",
    project="hand_bag_detector",
    name="yolov8n_v1",
    exist_ok=False,
    save=True,
    save_period=5,
    plots=True,
    verbose=True
)

print("\n" + "="*60)
print("TRAINING COMPLETED")
print("="*60)



STARTING MODEL TRAINING

Training configuration:
  • Model: YOLOv8n
  • Epochs: 50
  • Image size: 640
  • Batch size: 16
  • Early stopping patience: 10
  • Dataset YAML: /content/handbag.yaml

Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/handbag.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=

In [ ]:
from huggingface_hub import login
import getpass

print("\n" + "="*60)
print("🤗 HUGGING FACE AUTHENTICATION")

hf_token ="xxxxx"

# Login to Hugging Face
try:
    login(token=hf_token)
    print("\n  Logged in to Hugging Face successfully!")
except Exception as e:
    print(f"\n  Authentication failed: {str(e)}")
    raise


🤗 HUGGING FACE AUTHENTICATION

  Logged in to Hugging Face successfully!


In [12]:
from huggingface_hub import HfApi
from pathlib import Path

# =========================
# CONFIGURATION
# =========================

REPO_ID = "IndUSV/Yolov8_SE_3"

BEST_MODEL_PATH = Path(
    "/content/runs/detect/hand_bag_detector/yolov8n_v1/weights/best.pt"
)

# =========================
# INIT HF API
# =========================

api = HfApi()

print("\n" + "=" * 60)
print(" PUSHING TRAINED MODEL TO HUGGING FACE")
print("=" * 60)
print(f"\nTarget repository: {REPO_ID}\n")

# =========================
# CREATE / VERIFY REPO
# =========================

api.create_repo(
    repo_id=REPO_ID,
    repo_type="model",
    exist_ok=True,
    private=False
)

print("Repository ready")

# =========================
# CHECK MODEL
# =========================

if not BEST_MODEL_PATH.exists():
    raise FileNotFoundError(f"Model not found: {BEST_MODEL_PATH}")

print(f"Model found: {BEST_MODEL_PATH}")
print(f"Size: {BEST_MODEL_PATH.stat().st_size / (1024*1024):.2f} MB\n")

# =========================
# UPLOAD MODEL (CORRECT NAME)
# =========================

print("Uploading model weights...")

api.upload_file(
    path_or_fileobj=str(BEST_MODEL_PATH),
    path_in_repo="Yolov8_SE_2.pt",
    repo_id=REPO_ID,
    repo_type="model"
)

print("✅ Model uploaded successfully!")




 PUSHING TRAINED MODEL TO HUGGING FACE

Target repository: IndUSV/Yolov8_SE_3

Repository ready
Model found: /content/runs/detect/hand_bag_detector/yolov8n_v1/weights/best.pt
Size: 5.94 MB

Uploading model weights...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...olov8n_v1/weights/best.pt:   9%|8         |  533kB / 6.23MB            

✅ Model uploaded successfully!


In [13]:
from huggingface_hub import HfApi
from datetime import datetime

model_card_content = f"""
---
license: mit
tags:
- yolov8
- object-detection
- computer-vision
- mobile-phone
- suitcase
---

# YOLOv8n Multi-Object Detector (Phone & Suitcase)

A fine-tuned **YOLOv8 Nano (YOLOv8n)** model capable of detecting **mobile phones**, **suit cases** and **handbags**  in images.
The model has been trained and validated on custom datasets with diverse backgrounds, lighting conditions,
and object orientations.

## Model Details

- **Base Model**: YOLOv8n (Nano)
- **Task**: Object Detection
- **Supported Classes**:
  - `mobile_phone`
  - `suitcase`
- **Number of Classes**: 2
- **Input Resolution**: 640 × 640
- **Framework**: Ultralytics YOLO
- **Training Date**: {datetime.now().strftime('%Y-%m-%d')}

## Capabilities

This model can:
- Detect **mobile phones** in real-world scenes
- Detect **suitcases / luggage** in travel and indoor environments
- Handle multiple objects per image
- Perform robustly under varied lighting and viewpoints

## Training Details

- **Image Size**: 640
- **Optimizer**: Default YOLOv8 optimizer
- **Augmentations**: YOLOv8 default augmentations
- **Device**: NVIDIA GPU
- **Training Strategy**: Transfer learning from YOLOv8n

## Dataset Information

The model was trained using curated datasets containing:
- Mobile phone images with bounding-box annotations
- Suitcase / luggage images with bounding-box annotations
- Real-world indoor and outdoor scenes
- Multiple object instances per image

Dataset annotations follow object-detection standards compatible with YOLO training pipelines.

## Evaluation & Metrics

- Quantitative evaluation metrics are available in `results.csv`
- Qualitative predictions can be seen in validation images from the training run

## Files in This Repository

- `Yolov8_SE_2.pt` – Trained YOLOv8 model weights
- `args.yaml` – Training configuration
- `results.csv` – Training and validation metrics

## License

This model is released under the **MIT License**.

## Acknowledgements

- Ultralytics YOLOv8: https://github.com/ultralytics/ultralytics
- Training datasets sourced and curated for object detection research
"""

# Upload model card
api = HfApi()

print("\n  Creating and uploading model card...")
api.upload_file(
    path_or_fileobj=model_card_content.encode("utf-8"),
    path_in_repo="README.md",
    repo_id=REPO_ID,
    repo_type="model"
)

print("\n" + "=" * 60)
print("  MODEL CARD UPDATED SUCCESSFULLY")
print("=" * 60)
print(f"\n  Model URL: https://huggingface.co/{REPO_ID}")
print("  Capabilities: Mobile Phone Detection + Suitcase Detection")



  Creating and uploading model card...

  MODEL CARD UPDATED SUCCESSFULLY

  Model URL: https://huggingface.co/IndUSV/Yolov8_SE_3
  Capabilities: Mobile Phone Detection + Suitcase Detection



🧪 TESTING INFERENCE



FileNotFoundError: [Errno 2] No such file or directory: '/content/yolo_dataset/images/test'